<a href="https://colab.research.google.com/github/sayakpaul/Training-BatchNorm-and-Only-BatchNorm/blob/master/CIFAR10_Full_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Authenticate yourself to use the TPUs
import os

IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()

In [0]:
# TensorFlow Imports
import tensorflow as tf
print(tf.__version__)

2.2.0-rc3


In [0]:
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu) # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: grpc://10.120.151.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.120.151.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.120.151.66:8470']
Number of accelerators:  8


In [0]:
!pip install wandb -q
import wandb
wandb.login()

In [0]:
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/keras-idiomatic-programmer/master/zoo/resnet/resnet_cifar10.py

--2020-05-03 12:26:54--  https://raw.githubusercontent.com/GoogleCloudPlatform/keras-idiomatic-programmer/master/zoo/resnet/resnet_cifar10.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6064 (5.9K) [text/plain]
Saving to: ‘resnet_cifar10.py’

resnet_cifar10.py   100%[===================>]   5.92K  --.-KB/s    in 0s      

2020-05-03 12:26:54 (109 MB/s) - ‘resnet_cifar10.py’ saved [6064/6064]



In [0]:
# Other imports
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
import tensorflow as tf
import resnet_cifar10
import numpy as np
import time

# Random seed fixation
tf.random.set_seed(666)
np.random.seed(666)

In [0]:
def get_training_model():
    # ResNet20
    n = 2
    depth =  n * 9 + 2
    n_blocks = ((depth - 2) // 9) - 1

    # The input tensor
    inputs = Input(shape=(32, 32, 3))

    # The Stem Convolution Group
    x = resnet_cifar10.stem(inputs)

    # The learner
    x = resnet_cifar10.learner(x, n_blocks)

    # The Classifier for 10 classes
    outputs = resnet_cifar10.classifier(x, 10)

    # Instantiate the Model
    model = Model(inputs, outputs)

    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    
    return model

In [0]:
# Load the training set of CIFAR10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 3s 0us/step


In [0]:
BATCH_SIZE = 128 * strategy.num_replicas_in_sync

def normalize(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.cast(label, tf.int32)
    return image, label

def augment(image,label):
    image = tf.image.resize_with_crop_or_pad(image, 40, 40) # Add 8 pixels of padding
    image = tf.image.random_crop(image, size=[32, 32, 3]) # Random crop back to 32x32
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    image = tf.clip_by_value(image, 0., 1.)

    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = (
    train_ds
    .shuffle(1024)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = (
    test_ds
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [0]:
with strategy.scope():
    model = get_training_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_166 (Conv2D)             (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_164 (BatchN (None, 32, 32, 16)   64          conv2d_166[0][0]                 
__________________________________________________________________________________________________
re_lu_164 (ReLU)                (None, 32, 32, 16)   0           batch_normalization_164[0][0]    
____________________________________________________________________________________________

- Total params: 575,114
- **Trainable params: 571,114**
- Non-trainable params: 4,000


In [0]:
# Train model
wandb.init(project="training-bn-only", id="vanilla_resnet")

with strategy.scope():
    model = get_training_model()
    
start = time.time()
h = model.fit(train_ds,
         validation_data=test_ds,
         epochs=75,
         callbacks=[WandbCallback()])
end = time.time()
wandb.log({"training_time": end - start})
print("Network takes {:.3f} seconds to train".format(end - start))

Epoch 1/75
49/49 [==============================] - 10s 208ms/step - loss: 2.3432 - accuracy: 0.1359 - val_loss: 2.3748 - val_accuracy: 0.1517
Epoch 2/75
49/49 [==============================] - 5s 93ms/step - loss: 2.2274 - accuracy: 0.1759 - val_loss: 2.2666 - val_accuracy: 0.1680
Epoch 3/75
49/49 [==============================] - 5s 92ms/step - loss: 2.1778 - accuracy: 0.1971 - val_loss: 2.2408 - val_accuracy: 0.1646
Epoch 4/75
49/49 [==============================] - 5s 94ms/step - loss: 2.1347 - accuracy: 0.2145 - val_loss: 2.2012 - val_accuracy: 0.1790
Epoch 5/75
49/49 [==============================] - 5s 94ms/step - loss: 2.0963 - accuracy: 0.2235 - val_loss: 2.1505 - val_accuracy: 0.1908
Epoch 6/75
49/49 [==============================] - 5s 99ms/step - loss: 2.0715 - accuracy: 0.2345 - val_loss: 2.0890 - val_accuracy: 0.2148
Epoch 7/75
49/49 [==============================] - 5s 94ms/step - loss: 2.0469 - accuracy: 0.2429 - val_loss: 2.0506 - val_accuracy: 0.2346
Epoch 8/75


In [0]:
# Save the weights to GCS bucket for reproducibility
PROJECT_ID = "fast-ai-exploration" 
!gcloud config set project $PROJECT_ID

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [0]:
!gsutil mb gs://batch_norm_tpu

Creating gs://batch_norm_tpu/...


In [0]:
model.save("gs://batch_norm_tpu/vanilla_resnet")
# !gsutil -m cp -r vanilla_resnet gs://batch_norm_tpu

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: gs://batch_norm_tpu/vanilla_resnet/assets


INFO:tensorflow:Assets written to: gs://batch_norm_tpu/vanilla_resnet/assets


In [0]:
!gsutil ls -lh gs://batch_norm_tpu/vanilla_resnet/

       0 B  2020-05-03T12:35:08Z  gs://batch_norm_tpu/vanilla_resnet/
  4.94 MiB  2020-05-03T12:35:11Z  gs://batch_norm_tpu/vanilla_resnet/saved_model.pb
                                 gs://batch_norm_tpu/vanilla_resnet/assets/
                                 gs://batch_norm_tpu/vanilla_resnet/variables/
TOTAL: 2 objects, 5179967 bytes (4.94 MiB)


In [0]:
# Train model with a decay schedule
wandb.init(project="training-bn-only", id="resnet_lrsch_1")

first_decay_steps = 1000
lr_decayed_fn = (
  tf.keras.experimental.CosineDecay(
      initial_learning_rate=0.1,
      decay_steps=first_decay_steps))

with strategy.scope():
    model = get_training_model()
start = time.time()
h = model.fit(train_ds,
         validation_data=test_ds,
         epochs=75,
         callbacks=[WandbCallback()])
end = time.time()
wandb.log({"training_time": end - start})
print("Network takes {:.3f} seconds to train".format(end - start))

Epoch 1/75
49/49 [==============================] - 10s 213ms/step - loss: 2.3547 - accuracy: 0.1189 - val_loss: 2.3836 - val_accuracy: 0.1046
Epoch 2/75
49/49 [==============================] - 5s 95ms/step - loss: 2.2579 - accuracy: 0.1552 - val_loss: 2.2903 - val_accuracy: 0.1353
Epoch 3/75
49/49 [==============================] - 5s 94ms/step - loss: 2.2239 - accuracy: 0.1825 - val_loss: 2.2768 - val_accuracy: 0.1392
Epoch 4/75
49/49 [==============================] - 5s 93ms/step - loss: 2.1904 - accuracy: 0.1970 - val_loss: 2.2526 - val_accuracy: 0.1461
Epoch 5/75
49/49 [==============================] - 5s 95ms/step - loss: 2.1583 - accuracy: 0.2121 - val_loss: 2.2208 - val_accuracy: 0.1580
Epoch 6/75
49/49 [==============================] - 5s 95ms/step - loss: 2.1250 - accuracy: 0.2253 - val_loss: 2.1772 - val_accuracy: 0.1773
Epoch 7/75
49/49 [==============================] - 5s 96ms/step - loss: 2.0902 - accuracy: 0.2371 - val_loss: 2.1259 - val_accuracy: 0.2020
Epoch 8/75


In [0]:
model.save_weights("resnet_lrsch_1.h5")
!gsutil -m cp resnet_lrsch_1.h5 gs://batch_norm_tpu/

Copying file://resnet_lrsch_1.h5 [Content-Type=application/octet-stream]...
/ [1/1 files][  2.4 MiB/  2.4 MiB] 100% Done                                    
Operation completed over 1 objects/2.4 MiB.                                      


In [0]:
# Train model with a decay schedule
wandb.init(project="training-bn-only", id="resnet_lrsch_2")

first_decay_steps = 1000
lr_decayed_fn = (
  tf.keras.experimental.LinearCosineDecay(
      initial_learning_rate=0.1,
      decay_steps=first_decay_steps))

with strategy.scope():
    model = get_training_model()
start = time.time()
h = model.fit(train_ds,
         validation_data=test_ds,
         epochs=75,
         callbacks=[WandbCallback()])
end = time.time()
wandb.log({"training_time": end - start})
print("Network takes {:.3f} seconds to train".format(end - start))

Epoch 1/75
49/49 [==============================] - 11s 217ms/step - loss: 2.3147 - accuracy: 0.1224 - val_loss: 2.2760 - val_accuracy: 0.1195
Epoch 2/75
49/49 [==============================] - 5s 96ms/step - loss: 2.2221 - accuracy: 0.1673 - val_loss: 2.2651 - val_accuracy: 0.1324
Epoch 3/75
49/49 [==============================] - 5s 96ms/step - loss: 2.1814 - accuracy: 0.1892 - val_loss: 2.2502 - val_accuracy: 0.1333
Epoch 4/75
49/49 [==============================] - 5s 95ms/step - loss: 2.1487 - accuracy: 0.2052 - val_loss: 2.2196 - val_accuracy: 0.1400
Epoch 5/75
49/49 [==============================] - 5s 96ms/step - loss: 2.1181 - accuracy: 0.2160 - val_loss: 2.1744 - val_accuracy: 0.1585
Epoch 6/75
49/49 [==============================] - 5s 94ms/step - loss: 2.0934 - accuracy: 0.2263 - val_loss: 2.1275 - val_accuracy: 0.1840
Epoch 7/75
49/49 [==============================] - 5s 94ms/step - loss: 2.0672 - accuracy: 0.2372 - val_loss: 2.0858 - val_accuracy: 0.2020
Epoch 8/75


In [0]:
model.save_weights("resnet_lrsch_2.h5")
!gsutil -m cp resnet_lrsch_2.h5 gs://batch_norm_tpu/

Copying file://resnet_lrsch_2.h5 [Content-Type=application/octet-stream]...
/ [1/1 files][  2.4 MiB/  2.4 MiB] 100% Done                                    
Operation completed over 1 objects/2.4 MiB.                                      


In [0]:
with strategy.scope():
    model = get_training_model()

    for layer in model.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            if hasattr(layer, "trainable"):
                layer.trainable=False
    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_298 (Conv2D)             (None, 32, 32, 16)   448         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_284 (BatchN (None, 32, 32, 16)   64          conv2d_298[0][0]                 
__________________________________________________________________________________________________
re_lu_284 (ReLU)                (None, 32, 32, 16)   0           batch_normalization_284[0][0]    
____________________________________________________________________________________________

- Total params: 575,114
- **Trainable params: 4,000**
- Non-trainable params: 571,114

In [0]:
wandb.init(project="training-bn-only", id="bn-only-vanilla-1")

start = time.time()
h = model.fit(train_ds,
         validation_data=test_ds,
         epochs=75,
         callbacks=[WandbCallback()])
end = time.time()
wandb.log({"training_time": end - start})
print("Network takes {:.3f} seconds to train".format(end - start))

Epoch 1/75
49/49 [==============================] - 9s 189ms/step - loss: 2.4312 - accuracy: 0.0752 - val_loss: 2.5910 - val_accuracy: 0.0929
Epoch 2/75
49/49 [==============================] - 5s 93ms/step - loss: 2.4248 - accuracy: 0.0747 - val_loss: 2.3635 - val_accuracy: 0.0990
Epoch 3/75
49/49 [==============================] - 5s 93ms/step - loss: 2.4197 - accuracy: 0.0755 - val_loss: 2.3360 - val_accuracy: 0.0988
Epoch 4/75
49/49 [==============================] - 4s 81ms/step - loss: 2.4156 - accuracy: 0.0763 - val_loss: 2.3367 - val_accuracy: 0.0828
Epoch 5/75
49/49 [==============================] - 4s 79ms/step - loss: 2.4099 - accuracy: 0.0777 - val_loss: 2.3374 - val_accuracy: 0.0769
Epoch 6/75
49/49 [==============================] - 4s 79ms/step - loss: 2.4082 - accuracy: 0.0787 - val_loss: 2.3373 - val_accuracy: 0.0776
Epoch 7/75
49/49 [==============================] - 4s 78ms/step - loss: 2.3998 - accuracy: 0.0796 - val_loss: 2.3378 - val_accuracy: 0.0825
Epoch 8/75
4

In [0]:
model.save_weights("resnet_bn_only.h5")
!gsutil -m cp -r resnet_bn_only.h5 gs://batch_norm_tpu/

Copying file://resnet_bn_only.h5 [Content-Type=application/octet-stream]...
/ [1/1 files][  2.4 MiB/  2.4 MiB] 100% Done                                    
Operation completed over 1 objects/2.4 MiB.                                      


In [0]:
# Train model with a decay schedule
wandb.init(project="training-bn-only", id="bn-only-lrsch-1")

first_decay_steps = 1000
lr_decayed_fn = (
  tf.keras.experimental.CosineDecayRestarts(
      initial_learning_rate=0.1,
      first_decay_steps=first_decay_steps))

with strategy.scope():
    model = get_training_model()

    for layer in model.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            if hasattr(layer, "trainable"):
                layer.trainable=False
    model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(lr_decayed_fn), metrics=["accuracy"])
    
start = time.time()
h = model.fit(train_ds,
         validation_data=test_ds,
         epochs=75,
         callbacks=[WandbCallback()])
end = time.time()
wandb.log({"training_time": end - start})
print("Network takes {:.3f} seconds to train".format(end - start))

Epoch 1/75
49/49 [==============================] - 10s 200ms/step - loss: 2.4715 - accuracy: 0.0960 - val_loss: 2.3537 - val_accuracy: 0.1055
Epoch 2/75
49/49 [==============================] - 4s 91ms/step - loss: 2.3948 - accuracy: 0.1009 - val_loss: 2.3078 - val_accuracy: 0.1105
Epoch 3/75
49/49 [==============================] - 5s 92ms/step - loss: 2.3469 - accuracy: 0.1148 - val_loss: 2.2999 - val_accuracy: 0.1224
Epoch 4/75
49/49 [==============================] - 4s 91ms/step - loss: 2.3161 - accuracy: 0.1236 - val_loss: 2.2919 - val_accuracy: 0.1286
Epoch 5/75
49/49 [==============================] - 5s 93ms/step - loss: 2.2959 - accuracy: 0.1358 - val_loss: 2.2822 - val_accuracy: 0.1375
Epoch 6/75
49/49 [==============================] - 4s 92ms/step - loss: 2.2790 - accuracy: 0.1405 - val_loss: 2.2685 - val_accuracy: 0.1497
Epoch 7/75
49/49 [==============================] - 5s 92ms/step - loss: 2.2676 - accuracy: 0.1463 - val_loss: 2.2553 - val_accuracy: 0.1584
Epoch 8/75


In [0]:
model.save_weights("resnet_bn_only_2.h5")
!gsutil -m cp -r resnet_bn_only_2.h5 gs://batch_norm_tpu/

Copying file://resnet_bn_only_2.h5 [Content-Type=application/octet-stream]...
/ [1/1 files][  2.4 MiB/  2.4 MiB] 100% Done                                    
Operation completed over 1 objects/2.4 MiB.                                      
